# DELIV VR ASSIGNMENT
## Süleyman Topdemir

In [94]:
import pandas as pd
import numpy as np
from haversine import haversine
import copy as copy
import sys
import random
import time

# WARNING!!!!
Since I used the jupyter, the order and route objects must be created every time when we used them in the algorithms.

### reading the data to manipulate

to find better inital solutions, the sequence of the input data has been changed. If we use the raw order data, number of requests assigned one of the routes is less than the case sorted by the pickup window start.

In [95]:
#orderData = pd.read_excel (r'D:\\Users\\suuser\\Desktop\\Project Benjamin\\Take_home_deliveries_data.xlsx') # may need to be modified 
orderData = pd.read_excel (r'D:\\Users\\suuser\\Desktop\\Project Benjamin\\Take_home_deliveries_data_sortedByPickStart.xlsx')
orderDataFrame = pd.DataFrame(orderData, columns= ['Order_Id','Pickup_Window_Start','Pickup_Window_End',
                                  'Dropoff_Window_Start','Dropoff_Window_End','Pickup_Latitude','Pickup_Longitude'
                                  ,'Dropoff_Latitude','Dropoff_Longitude'])
driversData = pd.read_excel (r'D:\\Users\\suuser\\Desktop\\Project Benjamin\\Take_home_drivers_data.xlsx') # may need to be modified
driversDataFrame = pd.DataFrame(driversData, columns= ['Route_id','Route_Start_Time','Route_End_Time'])

a=orderDataFrame[['Pickup_Latitude','Pickup_Longitude','Dropoff_Latitude','Dropoff_Longitude']]

### Order Class

In [96]:
class Order(object):
    def __init__(self,orderId,ordertype,o_pick_start,o_pick_end,o_drop_start,o_drop_end,o_pickup_lat,o_pickup_long,o_drop_lat,o_drop_long,):
        self.orderId=orderId
        self.ordertype=ordertype
        self.o_pick_start=o_pick_start
        self.o_pick_end=o_pick_end
        self.o_drop_start=o_drop_start
        self.o_drop_end=o_drop_end
        self.o_pickup_lat=o_pickup_lat
        self.o_pickup_long=o_pickup_long
        self.o_drop_lat=o_drop_lat
        self.o_drop_long=o_drop_long
    # no need 
    def __eq__(self,other):
        return(self.__dict__==other.__dict__)
        

### Route Class

In [97]:
class Route(object):
    route_end_time=0
    pickupTime=10/60
    dropoffTime=5/60
    timePerDistance=5/60 #minutes
    # you need to think on these two. However, currently they are working well
    distanceBetweenTwoConsecutiveOrders=[]
    ArrivalTimesofTheOrdersIncludedWaste=[]
    # if a driver come to pickup an order early, needs to wait until the Pickup Window Start; waste time
    def __init__(self,route_id,route_start_time,route_end_time):
        self.route_id=route_id
        self.route_start_time=route_start_time
        self.route_end_time=route_end_time
        self.route_distance=0
        self.totalWasteWaiting=0
        self.orderSequence=[]
        
    def addOrdertoSequence(self,order):
        self.orderSequence.append(order)
    def removeOrderfromSequence(self,order):
        self.orderSequence.remove(order)
    # to calculate the total route distance by using a function which is outside of the class ..distanceFunction(o1,o2)
    def calculateDistancesAndTotalRouteDistance(self):
        self.route_distance=0
        self.distanceBetweenTwoConsecutiveOrders=[]
        if len(self.orderSequence)==0:
            print('this route has 0 element')
        else:
            for i in range(len(self.orderSequence)-1):
                self.distanceBetweenTwoConsecutiveOrders.append(distanceFunction(self.orderSequence[i],self.orderSequence[i+1]))
                self.route_distance=self.route_distance+distanceFunction(self.orderSequence[i],self.orderSequence[i+1])
    # we need the method below to calculate the arrival time of the orders in the route. Notice that, if the driver go to 
    # order point earlier than the start time of the time window, the truck must wait until the start time of the time window
    # this additional waiting time is expressed as 'totalWasteWaiting' below. 
    
    def DetermineArrivalTimesOfRequests(self):
        self.ArrivalTimesofTheOrdersIncludedWaste=[]
        self.totalWasteWaiting=0
        if len(self.orderSequence)==0:
            print('this route has 0 element')
        else:
            if self.route_start_time < self.orderSequence[0].o_pick_start:
                self.ArrivalTimesofTheOrdersIncludedWaste.append(self.orderSequence[0].o_pick_start)
                self.totalWasteWaiting=self.totalWasteWaiting+ (self.orderSequence[0].o_pick_start-self.route_start_time)
            else:
                self.ArrivalTimesofTheOrdersIncludedWaste.append(self.route_start_time)
            for i in range(len(self.orderSequence)-1):
                if self.orderSequence[i].ordertype=='p':
                    optionTravelPlusActTime=self.ArrivalTimesofTheOrdersIncludedWaste[-1]+self.pickupTime+self.distanceBetweenTwoConsecutiveOrders[i]*self.timePerDistance
                else:
                    optionTravelPlusActTime=self.ArrivalTimesofTheOrdersIncludedWaste[-1]+self.dropoffTime+self.distanceBetweenTwoConsecutiveOrders[i]*self.timePerDistance
                if optionTravelPlusActTime < ((self.orderSequence[i+1].o_pick_start)*(self.orderSequence[i+1].ordertype=='p'))+ ((self.orderSequence[i+1].o_drop_start)*(self.orderSequence[i+1].ordertype=='d')):
                    self.ArrivalTimesofTheOrdersIncludedWaste.append(((self.orderSequence[i+1].o_pick_start)*(self.orderSequence[i+1].ordertype=='p'))+((self.orderSequence[i+1].o_drop_start)*(self.orderSequence[i+1].ordertype=='d')))
                    self.totalWasteWaiting=self.totalWasteWaiting+(((self.orderSequence[i+1].o_pick_start)*(self.orderSequence[i+1].ordertype=='p'))+ ((self.orderSequence[i+1].o_drop_start)*(self.orderSequence[i+1].ordertype=='d'))-optionTravelPlusActTime)
                else:
                    self.ArrivalTimesofTheOrdersIncludedWaste.append(optionTravelPlusActTime) 
    # this method is defined not to use deepcopy() which takes relatively long time, really time-consuming
    def copyObject(self):
        w=Route(self.route_id,self.route_start_time,self.route_end_time)
        w.route_distance=self.route_distance
        w.totalWasteWaiting=self.totalWasteWaiting
        w.orderSequence=self.orderSequence[:]
        return(w) 
            

In [98]:
orderDataFrame.head()
#driversDataFrame.head()

,Order_Id,Pickup_Window_Start,Pickup_Window_End,Dropoff_Window_Start,Dropoff_Window_End,Pickup_Latitude,Pickup_Longitude,Dropoff_Latitude,Dropoff_Longitude
0,1,10,11,11,13,39.176461,-94.488092,39.167085,-94.501372
1,8,10,11,11,13,39.176461,-94.488092,39.318614,-94.503761
2,15,10,11,11,13,39.168461,-94.470092,39.197058,-94.470769
3,22,10,11,11,14,39.168461,-94.470092,39.162388,-94.546795
4,29,10,11,11,14,39.168461,-94.470092,39.280901,-94.365953


### to constitute the order objects

In [99]:
orderObjectList=[]
for index, row in orderDataFrame.iterrows():
    orderObjectList.append(Order(row['Order_Id'],'p',row['Pickup_Window_Start'],row['Pickup_Window_End'],
                           row['Dropoff_Window_Start'],row['Dropoff_Window_End'],row['Pickup_Latitude'],row['Pickup_Longitude'],
                           row['Dropoff_Latitude'],row['Dropoff_Longitude']))
    orderObjectList.append(Order(row['Order_Id'],'d',row['Pickup_Window_Start'],row['Pickup_Window_End'],
                           row['Dropoff_Window_Start'],row['Dropoff_Window_End'],row['Pickup_Latitude'],row['Pickup_Longitude'],
                           row['Dropoff_Latitude'],row['Dropoff_Longitude']))

#this part is for route objects

routeObjectList=[]
for index,row in driversDataFrame.iterrows():
    routeObjectList.append(Route(row['Route_id'],row['Route_Start_Time'],row['Route_End_Time']))
    


In [91]:
# to check 
a=orderObjectList[3]
print(a.__dict__)
print(a)
myNewObject=Order(a.orderId,a.ordertype,a.o_pick_start,a.o_pick_end,a.o_drop_start,a.o_drop_end,a.o_pickup_lat,a.o_pickup_long,
                 a.o_drop_lat,a.o_drop_long)
print(myNewObject)
print(myNewObject.__dict__)


{'orderId': 8.0, 'ordertype': 'd', 'o_pick_start': 10.0, 'o_pick_end': 11.0, 'o_drop_start': 11.0, 'o_drop_end': 13.0, 'o_pickup_lat': 39.176461, 'o_pickup_long': -94.488092, 'o_drop_lat': 39.3186136452268, 'o_drop_long': -94.5037607791607}
{'orderId': 8.0, 'ordertype': 'd', 'o_pick_start': 10.0, 'o_pick_end': 11.0, 'o_drop_start': 11.0, 'o_drop_end': 13.0, 'o_pickup_lat': 39.176461, 'o_pickup_long': -94.488092, 'o_drop_lat': 39.3186136452268, 'o_drop_long': -94.5037607791607}


### Function for Haversine Distance between two points

used a library called Haversine to calculate the distance between two coordinates

In [100]:
def distanceFunction(order1,order2):
    if order1.ordertype=='p' and order2.ordertype=='p':
        coordOrder1=(order1.o_pickup_lat,order1.o_pickup_long)
        coordOrder2=(order2.o_pickup_lat,order2.o_pickup_long)
        distance=haversine(coordOrder1,coordOrder2)
    elif order1.ordertype=='p' and order2.ordertype=='d':
        coordOrder1=(order1.o_pickup_lat,order1.o_pickup_long)
        coordOrder2=(order2.o_drop_lat,order2.o_drop_long)
        distance=haversine(coordOrder1,coordOrder2)
    elif order1.ordertype=='d' and order2.ordertype=='p':
        coordOrder1=(order1.o_drop_lat,order1.o_drop_long)
        coordOrder2=(order2.o_pickup_lat,order2.o_pickup_long)
        distance=haversine(coordOrder1,coordOrder2)
    else:
        coordOrder1=(order1.o_drop_lat,order1.o_drop_long)
        coordOrder2=(order2.o_drop_lat,order2.o_drop_long)
        distance=haversine(coordOrder1,coordOrder2)
    return distance

## Initial Routes

I have checked the literature for the construction heuristics and found some alternatives which are sequential and parallel construction heuristics. I chose the sequential one since this methods provides better results comparing with the parallel one.

the next 5 functions have been used for attaining the initials routes. In the given problem, the time window costraints can be classified as hard constraints. By using the sequential construction heuristics, I assign ~90/103 orders to the routes (all available ones). This process can be improved by using the "basicGreedyInsertion2" function which can be seen below. The input of the algorithm is the left orders (normally we use the orders achieved from randomRemovalHeuristic) and the routes achieved from the sequential construction heuristic.


the reason to use while(numberOfIterationsWithSameResult<len(routeObjList): is that I could not achieve zero left order while achieving the routes. The parallel cons approach can be tried, or existing approach can be enhanced

In [101]:
def SequentialConstructionHeuristic(orderObjList,routeObjList):
    filledRouteList=[]
    for item in routeObjList:
        filledRouteList.append(item)
    chosenRoute=-1
    numberOfIterationsWithSameResult=0
    while(numberOfIterationsWithSameResult<len(routeObjList)):
        firstLengthOftheOrderObjects=len(orderObjList)
        if chosenRoute==len(routeObjList)-1:
            chosenRoute=0
        else:
            chosenRoute=chosenRoute+1
        willRemoveList=[]
        for i in range(0,len(orderObjList),2):
            #filledRouteList[chosenRoute].addOrdertoSequence(orderObjList[i])
            #filledRouteList[chosenRoute].addOrdertoSequence(orderObjList[i+1])
            #if len(filledRouteList[chosenRoute].orderSequence)==0:
            #    print('chosen Route ', chosenRoute+1, 'has O elements ', 'i value is ',i  )
            filledRouteList[chosenRoute].orderSequence.append(orderObjList[i])
            filledRouteList[chosenRoute].orderSequence.append(orderObjList[i+1])
            #for item in filledRouteList[chosenRoute].orderSequence:
            #    print(item.orderId, item.ordertype, end="--")
            #print()
            #candidateRoute=copy.deepcopy(filledRouteList[chosenRoute])
            candidateRoute=filledRouteList[chosenRoute].copyObject()
            improvedCandidateRoute=HillClimbingRoutingAlgorithm(candidateRoute)
            if feasibilityControl(improvedCandidateRoute)==True:
                #filledRouteList[chosenRoute]=copy.deepcopy(improvedCandidateRoute)
                filledRouteList[chosenRoute]=improvedCandidateRoute.copyObject()
                willRemoveList.append(orderObjList[i+1])
                willRemoveList.append(orderObjList[i])
            else:
                #filledRouteList[chosenRoute].removeOrderfromSequence(orderObjList[i])
                #filledRouteList[chosenRoute].removeOrderfromSequence(orderObjList[i+1])
                del filledRouteList[chosenRoute].orderSequence[-1]
                del filledRouteList[chosenRoute].orderSequence[-1]
                #print()
                #for item in filledRouteList[chosenRoute].orderSequence:
                #    print(item.orderId, item.ordertype, end="--")
                #print("new order option")
        orderObjList=[x for x in orderObjList if x not in willRemoveList]
        secondLengthOftheOrderObjects=len(orderObjList)
        #for item in willRemoveList:
        #    print(item.orderId,item.ordertype,end="--")
        if firstLengthOftheOrderObjects==secondLengthOftheOrderObjects:
            numberOfIterationsWithSameResult=numberOfIterationsWithSameResult+1
        print(len(orderObjList))
    return(filledRouteList)
                
# this function is also used in our main algorithm
def feasibilityControl(can1Route):
    #canRoute=copy.deepcopy(can1Route)
    canRoute=can1Route.copyObject()
    feasibilityKey=True
    canRoute.calculateDistancesAndTotalRouteDistance()
    canRoute.DetermineArrivalTimesOfRequests()
    if canRoute.ArrivalTimesofTheOrdersIncludedWaste[-1]+(canRoute.orderSequence[-1].ordertype=='p')*(10/60)+(canRoute.orderSequence[-1].ordertype=='d')*(5/60) > canRoute.route_end_time:
        return(False)
    for i in range(len(canRoute.orderSequence)):
        if canRoute.orderSequence[i].ordertype=='p':
            if (canRoute.orderSequence[i].o_pick_start > canRoute.ArrivalTimesofTheOrdersIncludedWaste[i]) or (canRoute.orderSequence[i].o_pick_end < canRoute.ArrivalTimesofTheOrdersIncludedWaste[i]):
                feasibilityKey=False
                break
        else:
            if (canRoute.orderSequence[i].o_drop_start > canRoute.ArrivalTimesofTheOrdersIncludedWaste[i]) or (canRoute.orderSequence[i].o_drop_end < canRoute.ArrivalTimesofTheOrdersIncludedWaste[i]):
                feasibilityKey=False
                break
    return(feasibilityKey)  



In [102]:
def HillClimbingRoutingAlgorithm(aRoute):
    improvement=0
    key=True
    #bestRoute=copy.deepcopy(aRoute)
    bestRoute=aRoute.copyObject()
    #candidateRoute=copy.deepcopy(bestRoute)
    candidateRoute=bestRoute.copyObject()
    while improvement>0 or key==True:
        key=False
        #bestRoute=copy.deepcopy(candidateRoute)
        bestRoute=candidateRoute.copyObject()
        for i in range(len(bestRoute.orderSequence)):
            for j in range(i+1,len(bestRoute.orderSequence)):
                #candidateOfCandidateRoute=copy.deepcopy(bestRoute)
                candidateOfCandidateRoute=bestRoute.copyObject()
                # to check the deadline of the orders. 
                if ((bestRoute.orderSequence[j].o_pick_end*(bestRoute.orderSequence[j].ordertype=='p'))+
                (bestRoute.orderSequence[j].o_drop_end*(bestRoute.orderSequence[j].ordertype=='d')))<((bestRoute.orderSequence[i].o_pick_end*(bestRoute.orderSequence[i].ordertype=='p'))+
                (bestRoute.orderSequence[i].o_drop_end*(bestRoute.orderSequence[i].ordertype=='d'))) and typeFeasibilityControl(i,j,bestRoute):
                    x=candidateOfCandidateRoute.orderSequence[i]
                    candidateOfCandidateRoute.orderSequence[i]=candidateOfCandidateRoute.orderSequence[j]
                    candidateOfCandidateRoute.orderSequence[j]=x
                if costFunctionforHG(candidateOfCandidateRoute)<costFunctionforHG(candidateRoute):
                    #candidateRoute=copy.deepcopy(candidateOfCandidateRoute)
                    candidateRoute=candidateOfCandidateRoute.copyObject()
        improvement=costFunctionforHG(bestRoute)-costFunctionforHG(candidateRoute)
    return(bestRoute)           
# (total route duration*weight1) + (total number of time window violations in the route*weight2)
def costFunctionforHG(aRoute):
    # these values can be tuned
    weight1=0.9
    weight2=0.1
    aRoute.calculateDistancesAndTotalRouteDistance()
    aRoute.DetermineArrivalTimesOfRequests()
    totalDuration=aRoute.ArrivalTimesofTheOrdersIncludedWaste[-1]+ ((aRoute.orderSequence[-1].ordertype=='p')*aRoute.pickupTime)+((aRoute.orderSequence[-1].ordertype=='d')*aRoute.dropoffTime) - (aRoute.route_start_time)
    totalNumberofTWViolations=0
    for i in range(len(aRoute.orderSequence)):
        if aRoute.ArrivalTimesofTheOrdersIncludedWaste[i] > (aRoute.orderSequence[i].ordertype=='p')*aRoute.orderSequence[i].o_pick_start+(aRoute.orderSequence[i].ordertype=='d')*aRoute.orderSequence[i].o_drop_start:
            totalNumberofTWViolations=totalNumberofTWViolations+1
    totalCost=(weight1*totalDuration) + (weight2*totalNumberofTWViolations)
    #totalWasteWaiting
    return(totalCost)

def typeFeasibilityControl(firstIn,secondIn,route):
    feasKey=True
    #copyOftheRoute=copy.deepcopy(route)
    copyOftheRoute=route.copyObject()
    a=copyOftheRoute.orderSequence[firstIn]
    copyOftheRoute.orderSequence[firstIn]=copyOftheRoute.orderSequence[secondIn]
    copyOftheRoute.orderSequence[secondIn]=a
    for i in range(len(copyOftheRoute.orderSequence)):
        for j in range(i+1,len(copyOftheRoute.orderSequence)):
            if(copyOftheRoute.orderSequence[i].orderId==copyOftheRoute.orderSequence[j].orderId) and (copyOftheRoute.orderSequence[i].ordertype=='d'):
                feasKey=False
    return(feasKey)
            
    


The numbers you will see after executing the next cell are the number of left orders. It's decreasing at least 30. After that no order can be assigned to any route. To detect the orders which were not assigned to any route, I used the next function: findNonExistingOrders

In [103]:
fRouL=SequentialConstructionHeuristic(orderObjectList,routeObjectList)
print()
for objects2 in fRouL:
    print()
    for item in objects2.orderSequence:
        print(item.orderId, item.ordertype,end="")

194
178
164
152
136
120
110
94
78
68
58
52
46
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30


1.0 p8.0 p29.0 p1.0 d8.0 d29.0 d12.0 p6.0 p27.0 p6.0 d12.0 d27.0 d
15.0 p22.0 p36.0 p50.0 p15.0 d22.0 d50.0 d36.0 d75.0 p13.0 p20.0 p34.0 p75.0 d13.0 d20.0 d34.0 d
43.0 p57.0 p64.0 p57.0 d43.0 d2.0 p64.0 d2.0 d5.0 p19.0 p26.0 p19.0 d5.0 d26.0 d
71.0 p30.0 p37.0 p94.0 p71.0 d30.0 d37.0 d94.0 d7.0 p28.0 p7.0 d28.0 d
78.0 p78.0 d9.0 p23.0 p3.0 p23.0 d9.0 d3.0 d41.0 p48.0 p62.0 p48.0 d14.0 p62.0 d41.0 d14.0 d
85.0 p58.0 p72.0 p24.0 p31.0 p58.0 d85.0 d24.0 d72.0 d31.0 d55.0 p21.0 p49.0 p55.0 d21.0 d49.0 d
16.0 p10.0 p101.0 p16.0 d10.0 d101.0 d35.0 p56.0 p56.0 d35.0 d
44.0 p17.0 p38.0 p11.0 p18.0 p25.0 p44.0 d17.0 d96.0 p18.0 d38.0 d11.0 d25.0 d63.0 p96.0 d63.0 d
51.0 p45.0 p52.0 p59.0 p52.0 d45.0 d51.0 d59.0 d69.0 p76.0 p83.0 p77.0 p69.0 d83.0 d76.0 d77.0 d
66.0 p39.0 p46.0 p66.0 d39.0 d46.0 d42.0 p70.0 p42.0 d70.0 d
73.0 p74.0 p4.0 p33.0 p73.0 d74.0 d82.0 p4.0 d33.0 d82.0 d
32.0 p61.0 p103.0 p61.0 

In [105]:
def findNonExistingOrders(routeList,orderObjects):
    myList=[]
    newList=[]
    key=False
    for orderIds in orderObjects:
        key=False
        for item in routeList:
            for order in item.orderSequence:
                if orderIds.orderId == order.orderId:
                    key=True
        if key==False:
            myList.append(orderIds.orderId)
    newList=list(np.unique(myList))
    return(newList)


unusedOrderList=findNonExistingOrders(fRouL,orderObjectList)
print(unusedOrderList,sep="/n")
    

[47.0, 53.0, 54.0, 60.0, 65.0, 67.0, 68.0, 80.0, 81.0, 87.0, 88.0, 89.0, 93.0, 95.0, 102.0]


In [106]:
orderObjectList=[]
for index, row in orderDataFrame.iterrows():
    orderObjectList.append(Order(row['Order_Id'],'p',row['Pickup_Window_Start'],row['Pickup_Window_End'],
                           row['Dropoff_Window_Start'],row['Dropoff_Window_End'],row['Pickup_Latitude'],row['Pickup_Longitude'],
                           row['Dropoff_Latitude'],row['Dropoff_Longitude']))
    orderObjectList.append(Order(row['Order_Id'],'d',row['Pickup_Window_Start'],row['Pickup_Window_End'],
                           row['Dropoff_Window_Start'],row['Dropoff_Window_End'],row['Pickup_Latitude'],row['Pickup_Longitude'],
                           row['Dropoff_Latitude'],row['Dropoff_Longitude']))

#this part is for route objects

routeObjectList=[]
for index,row in driversDataFrame.iterrows():
    routeObjectList.append(Route(row['Route_id'],row['Route_Start_Time'],row['Route_End_Time']))
    


## LARGE NEIGH. SEARCH ALG

Random removal and basic greddy algorithms have been used for removal and insertion methods.

Some solutions with less requests than the previous one have been accepted by assigning a cost weight to the number of orders/requests which are not assigned. Total distance is another part of the total cost. The parameters are chosen roughly, so needs to be tuned. 

In [111]:
def LargeNeighborhoodSearch(iterNumber,initialRoute,percentageOfDestroy, orderObjects):
    
    currentRoute=copyListOfOrders(initialRoute)
    #currentRoute=initialRoute.copyObject()
    bestRoute=copyListOfOrders(currentRoute)
    iteration=0
    #timeout=time.time()+30 #the alg gonna end after 2 hours
    while iteration < iterNumber:
        print('iteration: ', iteration, 'best route objective: ', objectiveCostMethod(bestRoute),' length: ',len(findNonExistingOrders(bestRoute,orderObjects)))
        destroyedRoutes, destroyedOrders=randomRemovalMethod(currentRoute,percentageOfDestroy)
        unusedOrderList=findNonExistingOrders(currentRoute,orderObjects)
        destroyedAndUnusedOrders=list(set(destroyedOrders + unusedOrderList))
        leftOrders, temproraryRoute, feasibility= basicGreedyInsertion2(destroyedRoutes, destroyedAndUnusedOrders, orderObjects)
        print('number of left orders',len(leftOrders))
        #if feasibility==True and lengthControl(initialRoute)==lengthControl(temproraryRoute):
        currentRoute=copyListOfOrders(temproraryRoute)
        leftOrdersForBest=findNonExistingOrders(bestRoute,orderObjects)
        if objectiveCostMethod2(temproraryRoute,leftOrders)<= objectiveCostMethod2(bestRoute,leftOrdersForBest):
            bestRoute=copyListOfOrders(temproraryRoute)        
        iteration=iteration+1
    return(bestRoute)
    
# worst removal can be added
def randomRemovalMethod(currentRoute,percentageOfDestroy):
    curRoute=copyListOfOrders(currentRoute)
    print('removal')
    #curRoute=currentRoute.copyObject()
    numberofTotalOrders=0
    allOrderIdsInRoute=[]
    for item in curRoute:
        numberofTotalOrders=numberofTotalOrders+len(item.orderSequence)
        for orderx in item.orderSequence:
            allOrderIdsInRoute.append(orderx.orderId)
            #allOrderIdsInRoute.append(orderx)
    #print(*allOrderIdsInRoute, sep='\n')
    allOrderIdsInRouteNoRepeat=list(np.unique(allOrderIdsInRoute))  
    #print(*allOrderIdsInRouteNoRepeat, sep='\n')
    numberOfOrdersGonnaDestroy=round(numberofTotalOrders*percentageOfDestroy)
    destroyedOrders=random.sample(allOrderIdsInRouteNoRepeat,numberOfOrdersGonnaDestroy)
    #destroyedOrders=random.sample(allOrderIdsInRoute,numberOfOrdersGonnaDestroy)
    for destOrderx in destroyedOrders:
        for item in curRoute:
            for orderx in reversed(item.orderSequence):
                if orderx.orderId==destOrderx:
                    item.orderSequence.remove(orderx)
    # can be used for speeding up the alg
    if 1==2:      
        for item in curRoute:
            for orderx in item.orderSequence:
                for destOrderx in destroyedOrders:
                   #if orderx==destOrderx:
                    if orderx.orderId==destOrderx:
                        destroyedOrders.remove(destOrderx)
                        item.orderSequence.remove(orderx)
    return(curRoute,destroyedOrders)

# this is a simpler approach. we used the basicGreedyInsertion2 function. The only difference is that, we choose a removed order
# to insert a route respectively in basic greedy insertion. In 2, we check all the options, anf after that we choose the one
# with minimum cost. Third apprach can be regreting insertion which considers the difference best and second best options

def basicGreedyInsertion(destRoute, destOrdersIds, orderObjects): # simple one, by one order by at each iteration
    print('Basic greedy')
    noMoreRequestInserted=True
    feasibleRoutesFound=False
    #print('at first', len(destOrdersIds))
    while len(destOrdersIds)>0 and noMoreRequestInserted==True:
        print('length of left orders', len(destOrdersIds))
        for orderItem in reversed(destOrdersIds):
            firstLength=len(destOrdersIds)
            routeAndCosts=[]
            for routeItem in destRoute:
                feasibleSolutionAchieved,minimumCostValue,newCanRoute=findBestPositionandItsCostValue(orderItem,routeItem,orderObjects)
                costAndRoute={
                    'cost':minimumCostValue,
                    ########################################!!!!!!!!!!!!!!!!bak##########o kadar da önemli değil
                    #'canRoute':copy.deepcopy(newCanRoute),
                    'canRoute':newCanRoute.copyObject(),
                    'correspondingRouteIndex':destRoute.index(routeItem),
                    'feasible':feasibleSolutionAchieved,
                }
                routeAndCosts.append(costAndRoute)
            minCostValue=99999999999 
            feasibleOrNot=False
            for itemCostRouteAndIndex in routeAndCosts:
                if itemCostRouteAndIndex['cost']<minCostValue:
                    minCostValue=itemCostRouteAndIndex['cost']
                    candidate=itemCostRouteAndIndex['canRoute']
                    index=itemCostRouteAndIndex['correspondingRouteIndex']
                    feasibleOrNot=itemCostRouteAndIndex['feasible']
            if feasibleOrNot==True:
                #destRoute[index]=copy.deepcopy(candidate)
                destRoute[index]=candidate.copyObject()
                destOrdersIds.remove(orderItem)
        secondLenght=len(destOrdersIds)    
        if firstLength==secondLenght:
            noMoreRequestInserted=False
    if len(destOrdersIds)==0:
        feasibleRoutesFound=True       
    return(destRoute, feasibleRoutesFound)

# this function is used in greedy insertion
def findBestPositionandItsCostValue(orderNumber,route,orderObjects):
    #print('Basic greedy sub alg')
    ordersToInsert=[]
    canRouteAndCostValueList=[]
    feasibleSolutionAchieved=True
    for orderItem in orderObjects:
        if orderItem.orderId==orderNumber:
            ordersToInsert.append(orderItem)
    for i in range(len(route.orderSequence)+1):
        #print('i: ',i)
        for j in range(i+1,len(route.orderSequence)+2):
            #print('j: ',j)
            #candidateRoute=copy.deepcopy(route)
            candidateRoute=route.copyObject()
            #print("before")
            #print(candidateRoute.__dict__)
            candidateRoute.orderSequence.insert(i,ordersToInsert[0])
            candidateRoute.orderSequence.insert(j,ordersToInsert[1])
            #print('after modifying')
            #for orders in candidateRoute.orderSequence:
            #    print(orders.orderId, orders.ordertype,end="")
            #print()
            #candidateRoute.calculateDistancesAndTotalRouteDistance()
            #candidateRoute.DetermineArrivalTimesOfRequests()
            
            #print(candidateRoute.__dict__)
            candidateRoute.calculateDistancesAndTotalRouteDistance()
            candidateRoute.DetermineArrivalTimesOfRequests()
            if(feasibilityControl(candidateRoute)==True):
                
                candidateRouteAndCostValue={
                    'canRoute':candidateRoute,
                    'cost':candidateRoute.route_distance,
                }
                canRouteAndCostValueList.append(candidateRouteAndCostValue)
            else:
                candidateRouteAndCostValue={
                    'canRoute':candidateRoute,
                    'cost':99999999999,
                }
                canRouteAndCostValueList.append(candidateRouteAndCostValue)
    minCostValue=99999999999
    candRoute=route.copyObject()
    #candRoute=copy.deepcopy(route)
    for item in canRouteAndCostValueList:
        if item['cost']<=minCostValue:
            minCostValue=item['cost']
            candRoute=item['canRoute']
    if minCostValue == 99999999999:
        feasibleSolutionAchieved=False
    return(feasibleSolutionAchieved,minCostValue,candRoute)
             
# total objective cost values of all routes
def objectiveCostMethod(route):
    objectiveCost=0
    for item in route:
        objectiveCost=objectiveCost+item.route_distance
    return(objectiveCost) 

def objectiveCostMethod2(route,leftOrdersList):
    weightForTotalDistance=0.1
    weightForNumberOfUnusedOrders=0.5
    totalDistance=0
    for item in route:
        totalDistance=totalDistance+item.route_distance
    ObjectiveCost=weightForTotalDistance*totalDistance+weightForNumberOfUnusedOrders*len(leftOrdersList)
    return(ObjectiveCost)

def lengthControl(rou):
    totalLengthRou=0
    for item in rou:
        totalLengthRou=totalLengthRou+len(item.orderSequence)
    return(totalLengthRou)

def copyListOfOrders(routeList):
    copiedRouteList=[]
    for routeElement in routeList:
        copiedRouteList.append(routeElement.copyObject())
    return copiedRouteList
    

### Basic Greedy Insertion

In [109]:
def basicGreedyInsertion2(destRoute, destOrdersIds, orderObjects): 
    print('Basic greedy')
    noMoreRequestInserted=True
    feasibleRoutesFound=False
    #print('at first', len(destOrdersIds))
    while len(destOrdersIds)>0 and noMoreRequestInserted==True:
        itemRouteAndCostX=[]
        print('length of left orders', len(destOrdersIds))
        for orderItem in reversed(destOrdersIds):
            firstLength=len(destOrdersIds)
            routeAndCosts=[]
            for routeItem in destRoute:
                feasibleSolutionAchieved,minimumCostValue,newCanRoute=findBestPositionandItsCostValue(orderItem,routeItem,orderObjects)
                costAndRoute={
                    'cost':minimumCostValue,
                    ########################################!!!!!!!!!!!!!!!!bak##########o kadar da önemli değil
                    #'canRoute':copy.deepcopy(newCanRoute),
                    'canRoute':newCanRoute.copyObject(),
                    'correspondingRouteIndex':destRoute.index(routeItem),
                    'feasible':feasibleSolutionAchieved,
                }
                routeAndCosts.append(costAndRoute)
            minCostValue=99999999999 
            feasibleOrNot=False
            candidate=newCanRoute.copyObject()
            index=0
            for itemCostRouteAndIndex in routeAndCosts:
                if itemCostRouteAndIndex['cost']<minCostValue:
                    minCostValue=itemCostRouteAndIndex['cost']
                    candidate=itemCostRouteAndIndex['canRoute']
                    index=itemCostRouteAndIndex['correspondingRouteIndex']
                    feasibleOrNot=itemCostRouteAndIndex['feasible']
            orderIndexesRoutesCosts={
                'relatedOrderId':orderItem,
                'cost':minCostValue,
                'route':candidate.copyObject(),
                'routeIndex':index,
                'feasibility':feasibleOrNot
            }
            itemRouteAndCostX.append(orderIndexesRoutesCosts)
        costMin=99999999999
        feasC=False
        for element in itemRouteAndCostX:
            if element['cost']< costMin:
                orderIdXX=element['relatedOrderId']
                costMin=element['cost']
                chosenRoute=element['route']
                routeSIndex=element['routeIndex']
                feasC=element['feasibility']
        if feasC==True:
            destRoute[routeSIndex]=chosenRoute.copyObject()
            destOrdersIds.remove(orderIdXX)
        secondLenght=len(destOrdersIds)   
        if firstLength==secondLenght:
            noMoreRequestInserted=False
    if len(destOrdersIds)==0:
        feasibleRoutesFound=True       
    return(destOrdersIds, destRoute, feasibleRoutesFound)

In [112]:
orderObjectList=[]
for index, row in orderDataFrame.iterrows():
    orderObjectList.append(Order(row['Order_Id'],'p',row['Pickup_Window_Start'],row['Pickup_Window_End'],
                           row['Dropoff_Window_Start'],row['Dropoff_Window_End'],row['Pickup_Latitude'],row['Pickup_Longitude'],
                           row['Dropoff_Latitude'],row['Dropoff_Longitude']))
    orderObjectList.append(Order(row['Order_Id'],'d',row['Pickup_Window_Start'],row['Pickup_Window_End'],
                           row['Dropoff_Window_Start'],row['Dropoff_Window_End'],row['Pickup_Latitude'],row['Pickup_Longitude'],
                           row['Dropoff_Latitude'],row['Dropoff_Longitude']))


routeObjectList=[]
for index,row in driversDataFrame.iterrows():
    routeObjectList.append(Route(row['Route_id'],row['Route_Start_Time'],row['Route_End_Time']))
    


Execution

In [87]:
#def LargeNeighborhoodSearch(iterNumber,initialRoute,percentageOfDestroy, orderObjects):

LNOutput=LargeNeighborhoodSearch(10,fRouL,0.10, orderObjectList)

iteration:  0 best route objective:  823.9593011036338  length:  15
removal
Basic greedy
length of left orders 33
length of left orders 32
length of left orders 31
length of left orders 30
length of left orders 29
length of left orders 28
length of left orders 27
length of left orders 26
length of left orders 25
length of left orders 24
length of left orders 23
length of left orders 22
length of left orders 21
length of left orders 20
length of left orders 19
length of left orders 18
length of left orders 17
length of left orders 16
number of left orders 16
iteration:  1 best route objective:  801.3734879804665  length:  16
removal
Basic greedy
length of left orders 33
length of left orders 32
length of left orders 31
length of left orders 30
length of left orders 29
length of left orders 28
length of left orders 27
length of left orders 26
length of left orders 25
length of left orders 24
length of left orders 23
length of left orders 22
length of left orders 21
length of left orders 

length of left orders 14
length of left orders 13
number of left orders 13
iteration:  14 best route objective:  604.3927766144506  length:  13
removal
Basic greedy
length of left orders 31
length of left orders 30
length of left orders 29
length of left orders 28
length of left orders 27
length of left orders 26
length of left orders 25
length of left orders 24
length of left orders 23
length of left orders 22
length of left orders 21
length of left orders 20
length of left orders 19
length of left orders 18
length of left orders 17
length of left orders 16
length of left orders 15
length of left orders 14
number of left orders 14
iteration:  15 best route objective:  575.8923379814453  length:  14
removal
Basic greedy
length of left orders 32
length of left orders 31
length of left orders 30
length of left orders 29
length of left orders 28
length of left orders 27
length of left orders 26
length of left orders 25
length of left orders 24
length of left orders 23
length of left order

length of left orders 20
length of left orders 19
length of left orders 18
length of left orders 17
length of left orders 16
length of left orders 15
length of left orders 14
length of left orders 13
length of left orders 12
length of left orders 11
length of left orders 10
length of left orders 9
length of left orders 8
number of left orders 8
iteration:  28 best route objective:  559.6409629973518  length:  8
removal
Basic greedy
length of left orders 27
length of left orders 26
length of left orders 25
length of left orders 24
length of left orders 23
length of left orders 22
length of left orders 21
length of left orders 20
length of left orders 19
length of left orders 18
length of left orders 17
length of left orders 16
length of left orders 15
length of left orders 14
length of left orders 13
length of left orders 12
length of left orders 11
length of left orders 10
length of left orders 9
length of left orders 8
number of left orders 8
iteration:  29 best route objective:  559.

length of left orders 28
length of left orders 27
length of left orders 26
length of left orders 25
length of left orders 24
length of left orders 23
length of left orders 22
length of left orders 21
length of left orders 20
length of left orders 19
length of left orders 18
length of left orders 17
length of left orders 16
length of left orders 15
length of left orders 14
length of left orders 13
length of left orders 12
number of left orders 12
iteration:  42 best route objective:  559.6409629973518  length:  8
removal
Basic greedy
length of left orders 30
length of left orders 29
length of left orders 28
length of left orders 27
length of left orders 26
length of left orders 25
length of left orders 24
length of left orders 23
length of left orders 22
length of left orders 21
length of left orders 20
length of left orders 19
length of left orders 18
length of left orders 17
length of left orders 16
length of left orders 15
length of left orders 14
length of left orders 13
length of l

In [89]:
def writeTheResults(RouteObjectsList,orderObjects):
    
    routeIds=[]
    routeStartTimes=[]
    routeEndTimes=[]
    routeDistances=[]
    orderIds=[] 
    orderTypes=[] 
    orderPickupStartTime=[] 
    orderPickupEndTime=[]
    orderDropoffStartTime=[]
    orderDropoffEndTime=[]
    orderPickupLatitude=[]
    orderPickupLongitude=[]
    orderDropoffLatitude=[]
    orderDropoffLongitude=[]
    
    for element in RouteObjectsList:
        routeIds.append(element.route_id)
        routeStartTimes.append(element.route_start_time)
        routeEndTimes.append(element.route_end_time)
        routeDistances.append(element.route_distance)
        element.calculateDistancesAndTotalRouteDistance()
        element.DetermineArrivalTimesOfRequests()
        oID=[]
        oTy=[]
        oPST=[]
        oPET=[]
        oDST=[]
        oDET=[]
        oPLa=[]
        oPLo=[]
        oDLa=[]
        oDLo=[]
        for item in element.orderSequence:
            oID.append(item.orderId)
            oTy.append(item.ordertype)
            s=[i for i, n in enumerate(orderObjects) if n.orderId==item.orderId and n.ordertype=='p']
            oPST.append(orderObjects[s[0]].o_pick_start)
            oPET.append(orderObjects[s[0]].o_pick_end)
            oDST.append(orderObjects[s[0]].o_drop_start)
            oDET.append(orderObjects[s[0]].o_drop_end)
            oPLa.append(orderObjects[s[0]].o_pickup_lat)
            oPLo.append(orderObjects[s[0]].o_pickup_long)
            oDLa.append(orderObjects[s[0]].o_drop_lat)
            oDLo.append(orderObjects[s[0]].o_drop_long)        
        orderIds.append(oID)
        orderTypes.append(oTy)  
        orderPickupStartTime.append(oPST)
        orderPickupEndTime.append(oPET)
        orderDropoffStartTime.append(oDST)
        orderDropoffEndTime.append(oDET)
        orderPickupLatitude.append(oPLa)
        orderPickupLongitude.append(oPLo)
        orderDropoffLatitude.append(oDLa)
        orderDropoffLongitude.append(oDLo)
    
    with open("outPut.txt","w") as output:
        output.write("Route Ids-Route Start Times-Route End Times-Total Route Distances")
        output.write("\n")
        for i in range(len(routeIds)):
            output.write(str(routeIds[i])+"   "+str(routeStartTimes[i])+"   "+str(routeEndTimes[i])+"   "+str(routeDistances[i])+"\n")
 
        output.write("Order Ids")
        output.write("\n")
        for element in orderIds:
            output.write(str(element)+"\n")
            
        output.write("\n")
        output.write("Order Types")
        output.write("\n")
        for element in orderTypes:
            output.write(str(element)+"\n")
            
        output.write("\n")
        output.write("orderPickupStartTime")
        output.write("\n")
        for element in orderPickupStartTime:
            output.write(str(element)+"\n")
            
        output.write("\n")
        output.write("orderPickupEndTime")
        output.write("\n")
        for element in orderPickupEndTime:
            output.write(str(element)+"\n")
            
        output.write("\n")
        output.write("Arrival Times to pick and dropoff coordinates")
        output.write("\n")        
        for item in RouteObjectsList:
            output.write(str(item.ArrivalTimesofTheOrdersIncludedWaste)+"\n")
        
        output.write("\n")
        output.write("orderDropoffStartTime")
        output.write("\n")
        for element in orderDropoffStartTime:
            output.write(str(element)+"\n")
            
        output.write("\n")
        output.write("orderDropoffEndTime")
        output.write("\n")
        for element in orderDropoffEndTime:
            output.write(str(element)+"\n")  
            
        output.write("\n")
        output.write("orderPickupLatitude")
        output.write("\n")
        for element in orderPickupLatitude:
            output.write(str(element)+"\n") 
            orderPickupLongitude
        
        output.write("\n")
        output.write("orderPickupLongitude")
        output.write("\n")
        for element in orderPickupLongitude:
            output.write(str(element)+"\n")         
        
        output.write("\n")
        output.write("orderDropoffLatitude")
        output.write("\n")
        for element in orderDropoffLatitude:
            output.write(str(element)+"\n")
            
        output.write("\n")
        output.write("orderDropoffLongitude")
        output.write("\n")
        for element in orderDropoffLongitude:
            output.write(str(element)+"\n")            
        




In [91]:
writeTheResults(LNOutput,orderObjectList)

In [88]:
for objects2 in LNOutput:
    print()
    for item in objects2.orderSequence:
        print(item.orderId, item.ordertype,end="")
total=0
for j in LNOutput:
    total=total+len(j.orderSequence)
print('total',total)



22.0 p64.0 p51.0 p51.0 d64.0 d22.0 d4.0 p5.0 p33.0 p61.0 p4.0 d61.0 d5.0 d33.0 d
99.0 p43.0 p43.0 d37.0 p94.0 p101.0 p37.0 d99.0 d94.0 d101.0 d
15.0 p93.0 p23.0 p16.0 p9.0 p10.0 p15.0 d16.0 d9.0 d93.0 d23.0 d10.0 d
85.0 p86.0 p58.0 p58.0 d85.0 d86.0 d95.0 p102.0 p103.0 p95.0 d103.0 d102.0 d
57.0 p57.0 d38.0 p52.0 p45.0 p45.0 d52.0 d18.0 p11.0 p18.0 d40.0 p38.0 d40.0 d11.0 d63.0 p63.0 d
1.0 p2.0 p1.0 d2.0 d17.0 p31.0 p87.0 p17.0 d53.0 p54.0 p68.0 p87.0 d53.0 d54.0 d31.0 d68.0 d
24.0 p3.0 p3.0 d24.0 d69.0 p35.0 p35.0 d69.0 d
44.0 p80.0 p73.0 p81.0 p44.0 d46.0 p47.0 p73.0 d19.0 p26.0 p46.0 d47.0 d26.0 d80.0 d81.0 d7.0 p19.0 d7.0 d
25.0 p25.0 d6.0 p27.0 p62.0 p6.0 d27.0 d62.0 d
39.0 p88.0 p89.0 p82.0 p90.0 p39.0 d41.0 p56.0 p88.0 d89.0 d82.0 d90.0 d41.0 d56.0 d
59.0 p60.0 p74.0 p96.0 p60.0 d59.0 d97.0 p55.0 p48.0 p70.0 p77.0 p74.0 d48.0 d77.0 d55.0 d96.0 d97.0 d70.0 d
12.0 p34.0 p20.0 p12.0 d28.0 p84.0 p91.0 p98.0 p84.0 d20.0 d91.0 d28.0 d34.0 d98.0 d
75.0 p83.0 p76.0 p13.0 p21.0 p14.0 p4

In [ ]:
for objects2 in LNOutput:
    objects2.calculateDistancesAndTotalRouteDistance()
    objects2.DetermineArrivalTimesOfRequests()
print()

for objects2 in LNOutput:
    print()
    for item in objects2.orderSequence:
        print(item.orderId, item.ordertype,end="")
print()
print()


for objects2 in LNOutput:
    print(objects2.ArrivalTimesofTheOrdersIncludedWaste,sep="/n")
print()

for objects2 in LNOutput:
        print(objects2.distanceBetweenTwoConsecutiveOrders,sep="/n")
print()

the below part is for enhancing the initial solution quality

In [25]:
orderObjectList=[]
for index, row in orderDataFrame.iterrows():
    orderObjectList.append(Order(row['Order_Id'],'p',row['Pickup_Window_Start'],row['Pickup_Window_End'],
                           row['Dropoff_Window_Start'],row['Dropoff_Window_End'],row['Pickup_Latitude'],row['Pickup_Longitude'],
                           row['Dropoff_Latitude'],row['Dropoff_Longitude']))
    orderObjectList.append(Order(row['Order_Id'],'d',row['Pickup_Window_Start'],row['Pickup_Window_End'],
                           row['Dropoff_Window_Start'],row['Dropoff_Window_End'],row['Pickup_Latitude'],row['Pickup_Longitude'],
                           row['Dropoff_Latitude'],row['Dropoff_Longitude']))


routeObjectList=[]
for index,row in driversDataFrame.iterrows():
    routeObjectList.append(Route(row['Route_id'],row['Route_Start_Time'],row['Route_End_Time']))
    


In [59]:
for objects2 in fRouL:
    objects2.calculateDistancesAndTotalRouteDistance()
    objects2.DetermineArrivalTimesOfRequests()
print()

for objects2 in fRouL:
    print()
    for item in objects2.orderSequence:
        print(item.orderId, item.ordertype,end="")
print()
print()


for objects2 in fRouL:
    print(objects2.ArrivalTimesofTheOrdersIncludedWaste,sep="/n")
print()

for objects2 in fRouL:
        print(objects2.distanceBetweenTwoConsecutiveOrders,sep="/n")
print()




1.0 p8.0 p29.0 p1.0 d8.0 d29.0 d12.0 p6.0 p27.0 p6.0 d12.0 d27.0 d
15.0 p22.0 p36.0 p50.0 p15.0 d22.0 d50.0 d36.0 d75.0 p13.0 p20.0 p34.0 p75.0 d13.0 d20.0 d34.0 d
43.0 p57.0 p64.0 p57.0 d43.0 d2.0 p64.0 d2.0 d5.0 p19.0 p26.0 p19.0 d5.0 d26.0 d
71.0 p30.0 p37.0 p94.0 p71.0 d30.0 d37.0 d94.0 d7.0 p28.0 p7.0 d28.0 d
78.0 p78.0 d9.0 p23.0 p3.0 p23.0 d9.0 d3.0 d41.0 p48.0 p62.0 p48.0 d14.0 p62.0 d41.0 d14.0 d
85.0 p58.0 p72.0 p24.0 p31.0 p58.0 d85.0 d24.0 d72.0 d31.0 d55.0 p21.0 p49.0 p55.0 d21.0 d49.0 d
16.0 p10.0 p101.0 p16.0 d10.0 d101.0 d35.0 p56.0 p56.0 d35.0 d
44.0 p17.0 p38.0 p11.0 p18.0 p25.0 p44.0 d17.0 d96.0 p18.0 d38.0 d11.0 d25.0 d63.0 p96.0 d63.0 d
51.0 p45.0 p52.0 p59.0 p52.0 d45.0 d51.0 d59.0 d69.0 p76.0 p83.0 p77.0 p69.0 d83.0 d76.0 d77.0 d
66.0 p39.0 p46.0 p66.0 d39.0 d46.0 d42.0 p70.0 p42.0 d70.0 d
73.0 p74.0 p4.0 p33.0 p73.0 d74.0 d82.0 p4.0 d33.0 d82.0 d
32.0 p61.0 p103.0 p61.0 d32.0 d103.0 d
40.0 p90.0 p97.0 p90.0 d40.0 d97.0 d
92.0 p99.0 p92.0 d79.0 p86.0 p100.0 p79

In [29]:
for objects2 in fRouL:
    print()
    for item in objects2.orderSequence:
        print(item.orderId, item.ordertype,end="")
print()
for objects2 in maybeInitialFull:
    print()
    for item in objects2.orderSequence:
        print(item.orderId, item.ordertype,end="")        



1.0 p8.0 p29.0 p1.0 d8.0 d29.0 d12.0 p6.0 p27.0 p6.0 d12.0 d27.0 d
15.0 p22.0 p36.0 p50.0 p15.0 d22.0 d50.0 d36.0 d75.0 p13.0 p20.0 p34.0 p75.0 d13.0 d20.0 d34.0 d
43.0 p57.0 p64.0 p57.0 d43.0 d2.0 p64.0 d2.0 d5.0 p19.0 p26.0 p19.0 d5.0 d26.0 d
71.0 p30.0 p37.0 p94.0 p71.0 d30.0 d37.0 d94.0 d7.0 p28.0 p7.0 d28.0 d
78.0 p78.0 d9.0 p23.0 p3.0 p23.0 d9.0 d3.0 d41.0 p48.0 p55.0 p90.0 p48.0 d55.0 d41.0 d90.0 d
85.0 p58.0 p72.0 p24.0 p31.0 p58.0 d85.0 d24.0 d72.0 d31.0 d62.0 p14.0 p62.0 d14.0 d
16.0 p10.0 p101.0 p16.0 d10.0 d101.0 d21.0 p35.0 p21.0 d35.0 d
44.0 p17.0 p38.0 p11.0 p18.0 p25.0 p44.0 d17.0 d96.0 p18.0 d38.0 d11.0 d25.0 d63.0 p96.0 d63.0 d
51.0 p45.0 p52.0 p59.0 p52.0 d45.0 d51.0 d59.0 d69.0 p76.0 p83.0 p49.0 p77.0 p83.0 d69.0 d49.0 d76.0 d77.0 d
66.0 p39.0 p74.0 p66.0 d39.0 d74.0 d42.0 p56.0 p42.0 d56.0 d
73.0 p81.0 p4.0 p33.0 p73.0 d81.0 d47.0 p4.0 d33.0 d47.0 d
32.0 p61.0 p103.0 p61.0 d32.0 d103.0 d
40.0 p97.0 p70.0 p40.0 d97.0 d70.0 d
92.0 p99.0 p92.0 d79.0 p86.0 p100.0 p79.